In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from keras.optimizers import Adam


In [2]:
img_height = 300
img_width = 250
batch_size = 4
input_shape = (img_height,img_width,3)
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Specify the training data directory
train_data = train_gen.flow_from_directory(
    directory='data\cropped_3_classes\Train',
    shuffle=True,
    batch_size=batch_size,
    class_mode='categorical',
    seed = 42
)



validation_data = test_gen.flow_from_directory(
    directory='data\cropped_3_classes\Test',
    shuffle=False,
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

# Define the inputs
inputs = keras.Input(shape=input_shape)

# Load the MobileNetV2 model
base_model = ResNet50(weights='imagenet',
                         include_top=False,
                         input_shape=(img_height, img_width, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Pass the resized inputs through the base model
x = base_model(inputs)

# Add additional layers
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(3, activation='softmax')(x)

# Create the final model
my_model = Model(inputs=inputs, outputs=predictions)

# Print the model summary
my_model.summary()


In [8]:
from tensorflow.keras.callbacks import EarlyStopping,BackupAndRestore,ModelCheckpoint

cp = keras.callbacks.ModelCheckpoint(
    filepath='Results\ResNet50\cp',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


bc = keras.callbacks.BackupAndRestore(
    'Results\ResNet50\wbc', save_freq="epoch", delete_checkpoint=True)


es = EarlyStopping(patience=5, monitor='val_accuracy', verbose=1,restore_best_weights=True)

In [9]:
from tensorflow.keras.metrics import *

metrics = ['accuracy', Precision(), Recall(), AUC()]

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

my_model.compile(loss=CategoricalCrossentropy(),
                 optimizer=Adam(learning_rate=0.001),
                 metrics=metrics)

In [ ]:
history = my_model.fit(
    train_data,epochs=1000,
    validation_data=validation_data,
    callbacks=[es,cp,bc])


In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()


In [ ]:
my_model.save('ResNet_pretrained.h5')